In [1]:
import pandas as pd
import sys
sys.path.insert(0,'..')
import matplotlib.pyplot as plt
import os.path
import acquire_sso, prepare_sso
import prepare

plt.rc('figure', figsize=(16,10))
plt.rc('font', size=14)
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_columns', 50)

In [2]:
if os.path.isfile('sso_dict.csv'):
    df_dict = pd.read_csv('sso_dict.csv')
else:
    url_sso_dict = '''https://storage.googleapis.com/sa_saws_data/SAWS_SSO_DataFieldDescription_MM.xlsx'''
    df_dict = pd.read_excel(url_sso_dict)
    df_dict.to_csv('sso_dict.csv', index=False)

In [3]:
df_dict

,Field,Data Description
0,SSO_ID,Internal ID
1,INSPKEY,Not Used
2,SERVNO,Service Req # (internal use only)
3,REPORTDATE,Date Reported
4,SPILL_ADDRESS,Street number of spill
5,SPILL_ST_NAME,Streen name of spill
6,TOTAL_GAL,Total gallons spilled
7,GALSRET,Gallons returned to collection system
8,GAL,Disregard
9,SPILL_START,When SAWS received information about the spill or arrived at the location


In [4]:
# df = acquire_sso.acquire_sso()

In [5]:
# df = prepare_sso.prepare_sso_df()
# df.head(1)

In [6]:
df = prepare.prepare_sso_df()
print(df.shape[0])
df.head(1)

AttributeError: 'DataFrame' object has no attribute 'SPILL_START'

In [ ]:
# df.columns = ['sso_id','report_date','spill_address_num','spill_st_name',
#  'total_gal','gals_ret','spill_start','spill_stop','hrs','cause',
#  'comments','actions','watershed','unit_id','unit_id2','discharge_to',
#  'discharge_route','council_district','month','year','week',
#  'earz_zone','pipe_diam','pipe_len','pipe_type','inst_year','inches_no',
#  'rainfall_last3','spill_address_full','num_spills_recorded',
#  'num_spills_24mos','prevspill_24mos','unit_type','asset_type',
#  'last_cleaned','response_time','response_dttm','public_notice',
#  'root_cause','hrs_2','gal_2','hrs_3','gal_3','days_since_cleaned']

In [ ]:
# df.head(1)

In [ ]:
# df.ResponseTime * 60

In [ ]:
df.shape

In [ ]:
df['SPILL ADDRESS']

In [ ]:
# df_dict.shape

In [ ]:
# df_dict['Data Description'].value_counts()

In [ ]:
# unused = ['Disregard','Ignore','Service Req # (internal use only)',
#           'Not Used','Old mapping system reference (internal only)']
# df_dict[df_dict['Data Description'].isin(unused)]

In [ ]:
# ready1_dict = df_dict[~df_dict['Data Description'].isin(unused)]
# print(len(ready1_dict))
# ready1_dict

In [ ]:
# null_fields = ['SPILL_START_2','SPILL_START_3',
#                'SPILL_STOP_2','SPILL_STOP_3']
# final_dict = ready1_dict[~ready1_dict.Field.isin(null_fields)]\
#                     .reset_index(drop=True)
# len(final_dict)

In [ ]:
# final_dict

In [ ]:
# features_to_use = list(final_dict.Field)
# len(features_to_use)

In [ ]:
# bad_features = list(df.columns[~df.columns.isin(features_to_use)])

In [ ]:
# df = df.drop(columns=bad_features)
# df = df.drop(columns = ['TIMEINT','STEPS_TO_PREVENT'])
# df.shape

In [ ]:
# string_features = ['SSO_ID','SPILL_ADDRESS','COUNCIL_DISTRICT',]
# for col in string_features:
#     df[col] = df[col].astype(str)
    
# time_features = ['REPORTDATE','SPILL_START','SPILL_STOP',
#                  'ResponseDTTM', 'LASTCLND']
# for col in time_features:
#     df[col] = pd.to_datetime(df[col])
    
# fill_features = ['NUM_SPILLS_24MOS','PREVSPILL_24MOS','HRS_2',
#                 'HRS_3','GAL_2','GAL_3']
# for col in fill_features:
#     df[col] = df[col].fillna(0)
    
# df.Root_Cause = df.Root_Cause.str.strip()

In [ ]:
# df['days_since_cleaned'] = (df.SPILL_START - df.LASTCLND).dt.days

In [ ]:
df[df.days_since_cleaned == df.days_since_cleaned.min()]

In [ ]:
features = df.isna().sum() / df.shape[0]
bad_list = features[features > .5]
print(len(bad_list))
bad_list

In [ ]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))

In [ ]:
df.columns = df.columns.str.lower()
df.head(1)

In [ ]:
len(df.REPORTDATE.unique())

In [ ]:
df[df.NUM_SPILLS_COMPKEY == df.NUM_SPILLS_COMPKEY.max()]

In [ ]:
df[df.TOTAL_GAL.isin(list(df.TOTAL_GAL.nlargest(8)))]

In [ ]:
df.Root_Cause.value_counts()

In [ ]:
df.INSTYEAR.dropna().astype(int).value_counts().head(10).plot.bar()
plt.xticks(rotation=45)

plt.ylabel('Count of year installed')
plt.xlabel('Year')
plt.title('Top 10 years installed')
plt.show()

In [ ]:
df.COMMENTS.value_counts().head(10).plot.barh()